In [1]:
from tree_sitter import Language, Parser


In [2]:
def walk_tree(node, source_bytes, level=0):
    """
    Recursively walks the Abstract Syntax Tree (AST) and prints nodes.
    """
    indent = "  " * level
    
    # Extract the actual text for this node
    node_text = source_bytes[node.start_byte:node.end_byte].decode('utf-8').strip()
    
    # Only print relevant nodes (skip pure whitespace if desired)
    if node_text:
        # Print format: [Node Type] : Content
        print(f"{indent}[{node.type}] : {node_text.splitlines()}...")

    # Recurse for children
    for child in node.children:
        walk_tree(child, source_bytes, level + 1)


In [ ]:
# 1. Load the compiled language
# For tree-sitter >= 0.25.0, we need to use ctypes to load the library
import ctypes

# Load the shared library
lib = ctypes.CDLL('build/siasun_robot.dylib')

# Get the language function - must match the grammar name
tree_sitter_siasun_robot = lib.tree_sitter_siasun_robot
tree_sitter_siasun_robot.restype = ctypes.c_void_p

# Create Language object with the pointer from the library
SIASUN_LANG = Language(tree_sitter_siasun_robot())

TypeError: __init__() takes exactly 1 argument (2 given)

In [ ]:
# 2. Initialize the parser
parser = Parser(SIASUN_LANG)

# 3. Define a sample Siasun Robot Program
# Based on the manual:
# - NOP/END structure [1]
# - MOVJ/MOVL motion commands [2, 3]
# - SET assignment [4]
# - IF logic and Labels [5, 6]
# - I/O Control [7]
source_code = b"""
NOP

// Initialize Variables
SET I1 0
SET R1 10.5

// Label Definition
L10

// Motion Instructions
MOVJ P1 VJ=50 ACC=100 CNT=0
MOVL P2 VL=600 ACC=50 CNT=1

// IO Operation
OUT OT#1=ON

// Logic Control
ADD I1 1
IF I1 < 5 L10

// Conditional Call
IF I1 = 5 CALL SUB1

END
"""

# 4. Parse the code
tree = parser.parse(source_code)
root_node = tree.root_node

# 5. Visualize the Syntax Tree
print(f"Root Type: {root_node.type}")
print("-" * 40)

# Recursive function to traverse and print the tree
walk_tree(root_node, source_code)